In [1]:
%matplotlib widget
from huffTreeUtilities import * #TODO RECOPY THESE FILES
import numpy as np
import sys
import pandas as pd

In [2]:
I = loadHuffableImage('happyFace.png')
encoder, decoder = buildHuffPair(I)
#print(I.ravel())
enI = ''.join(encoder[pix] for pix in I.ravel())
allbins = np.arange(257)



loadHuffableImage: input is multi-channel, using grayscale.
loadHuffableImage: Setting range to [0, 255]


In [3]:
def decode(encoded):
    out = []
    while not encoded == "": 
        i = 1
        cut = encoded[:i]
        while cut not in decoder.keys():
            i = i + 1
            cut = encoded[:i]
        out.append(decoder[cut].astype(np.uint8))
        encoded = encoded[i:]
    return out

In [4]:
#print(t)
len(enI)
out = decode(enI)

In [5]:
len(out)
I.size
reconstuction = np.array(out)
reconstuction = reconstuction.reshape(I.shape)
old_out = reconstuction

In [6]:
plt.figure()
plt.imshow(reconstuction, cmap="hot")
print((reconstuction == I.all()).any())
print((reconstuction == I.all()).any())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

True
True


In [7]:
I = plt.imread("earth2.jpg")
encoder, decoder = buildHuffPair(I)
enI = ''.join(encoder[pix] for pix in I.ravel())

out = decode(enI)
reconstuction = np.array(out)
reconstuction = reconstuction.reshape(I.shape)
old_out = reconstuction

In [8]:
plt.figure()
plt.title("reconstuction")
plt.imshow(reconstuction)
print((reconstuction == I.all()).any())
print((reconstuction == I.all()).any())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

True
True


In [9]:
def getCompressionStats(fileName):
    I = plt.imread(fileName)
    if(type(I[0,0,0]) == np.float32):
        I = (I * 255)  
    print(fileName , "is", I.nbytes, "bytes")
    
    colors = ["Red", "Blue", "Green"]
    for chan in range(3):
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print(fileName + "'s", colors[chan], "channle is", I.nbytes, "bytes")
        allbins = np.unique(I[...,chan].ravel())
        encoder, decoder = buildHuffPair(I[...,chan])
        enI = ''.join(encoder[pix] for pix in I[...,chan].ravel())
        compSize = sys.getsizeof(decoder)+(len(enI)/8) #pretending that enI is a bitArray
        huffBitPixel = (compSize*8)/(I.shape[0]*I.shape[1]) #bits per pixle
        print("compressed image's", colors[chan] ,"channle is",  compSize ,"bytes or", huffBitPixel, "bits per pixel")
        freqs = np.histogram(I[...,chan],allbins)
        probs = freqs[0]/freqs[0].sum() # probabilities 
        print(colors[chan] ,"channel entropy is ",-(probs * np.log2(probs)).sum(), "bits/pixel")
        


In [10]:
earthEnc = getCompressionStats("earth2.jpg")
print("-------------------------- \n \n")
faceEnc = getCompressionStats("happyFace.png")


earth2.jpg is 307200 bytes
++++++++++++++++++++++++++++++++++++++++++++++++++++
earth2.jpg's Red channle is 307200 bytes
compressed image's Red channle is 86020.375 bytes or 6.720341796875 bits per pixel
Red channel entropy is  5.947979586232471 bits/pixel
++++++++++++++++++++++++++++++++++++++++++++++++++++
earth2.jpg's Blue channle is 307200 bytes
compressed image's Blue channle is 86416.125 bytes or 6.751259765625 bits per pixel
Blue channel entropy is  5.981984371723275 bits/pixel
++++++++++++++++++++++++++++++++++++++++++++++++++++
earth2.jpg's Green channle is 307200 bytes
compressed image's Green channle is 88024.5 bytes or 6.8769140625 bits per pixel
Green channel entropy is  6.112037905012562 bits/pixel
-------------------------- 
 

happyFace.png is 6553440 bytes
++++++++++++++++++++++++++++++++++++++++++++++++++++
happyFace.png's Red channle is 6553440 bytes
compressed image's Red channle is 164804.875 bytes or 2.4141928513879733 bits per pixel
Red channel entropy is  1.5767

In [11]:

#faceDecoder
def build_df(filename):
    I = loadHuffableImage(filename)
    encoder, decoder = buildHuffPair(I)
    faceFreq, bins = np.histogram(I.ravel(), np.arange(257))
    df = pd.DataFrame(decoder.items(), columns = ["decoding","intensitys"])#np.arange(256))
    df['#bits'] = df.iloc[:,0].apply(len)
    df['intensitys'] = pd.to_numeric(df['intensitys'])
    df["decoding"] = df["decoding"].astype(str)
    df = df.set_index("intensitys")
    df_f = pd.DataFrame(faceFreq)#[df.index]
    df_f.iloc[df.index]
    df["freqs"] = df_f
    df = df.sort_values("intensitys")
    return df, I, encoder, decoder

In [12]:
#df_happy, encoder, decoder = build_df("happyFace.png")
#print(df_happy.sort_values(by='#bits').head(20).sort_values(by='intensitys'))
#taking the 20 lowest bits and then sorting by intensity to find pairs
#I could turn all 214's into 217'2 saving 2bit for each 16259 time so total savings of 3.954KB
df_earth, I, encoder, decoder = build_df("earth2.jpg")
print(df_earth.sort_values(by='#bits').head(20).sort_values(by='intensitys'))
#if we turn all the 1's into 0's we save 4bits each 1666 times we save 866 bytes total


loadHuffableImage: input is multi-channel, using grayscale.
            decoding  #bits  freqs
intensitys                        
0                 11      2  35525
1             100101      6   1666
2            0010010      7    549
23           0010101      7    552
24           0010110      7    554
25           0011110      7    591
26           0010111      7    559
27           0010001      7    547
28           0001000      7    502
29           0000111      7    502
31           0000110      7    500
62           0000101      7    497
64           0001111      7    536
66           0001101      7    520
70           0000011      7    494
71           0010100      7    551
73           0000000      7    483
74           0000001      7    486
114         01100101      8    351
115         01000110      8    310


In [13]:
encoder[1] = encoder[0]
enI = ''.join(encoder[pix] for pix in I.ravel())
out = decode(enI)
reconstuction = np.array(out)
reconstuction = reconstuction.reshape(I.shape)

In [14]:
#fig, ax = plt.subplots(1,2,sharex=True,sharey=True)
#ax[0].imshow(reconstuction)
#ax[0].set_title("replaced intensity")
#ax[1].imshow(loadHuffableImage("earth2.jpg"))
#ax[1].set_title("raw")

In [15]:
def compress(group_size = 5):
    df_earth, I, encoder, decoder = build_df("earth2.jpg")
    lst = [df_earth.iloc[i:i+group_size] for i in range(0,len(df_earth)-group_size+1,group_size)]
    for group in range(len(lst)):
    #    print("idx with max freq:",lst[group].freqs.idxmax())
        x = lst[group].freqs.idxmax()
    #    print("Updating all other all to max freqs encoding", lst[group].iloc[lst[group].index != x,:].index)
        encoder.update({i : lst[group].decoding.loc[x] for i in lst[group].iloc[lst[group].index != x,:].index})
        enI = ''.join(encoder[pix] for pix in I.ravel())

    out = decode(enI)
    reconstuction = np.array(out)
    reconstuction = reconstuction.reshape(I.shape)
    
    return reconstuction

In [21]:
fig, ax = plt.subplots(1,3,sharex=True,sharey=True)
fig2, ax2 = plt.subplots(1,1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:

group_size = 30
rec = compress(group_size)
ax[0].imshow(rec)
st = "replaced intensity groups of" + str(group_size)
ax[0].set_title(st)
ax[1].imshow(loadHuffableImage("earth2.jpg"))
ax[1].set_title("raw")
ax[2].imshow(loadHuffableImage("earth2.jpg") - rec)
ax[2].set_title("raw-replaced")
n, bins, pat = ax2.hist(rec.ravel(),range(256))

loadHuffableImage: input is multi-channel, using grayscale.
loadHuffableImage: input is multi-channel, using grayscale.
loadHuffableImage: input is multi-channel, using grayscale.


In [ ]:
np.array([n != 0]).sum()
255/51
#proff that this my histogram is right